In [7]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

# Definir as classes dos sons a serem aprendidos
classes = ['Crying_LimpoV3', 'Laugh_LimpoV3', 'Noise_LimpoV3']
numero_de_mfccs = 100  # Número de coeficientes MFCCs a serem extraídos

# Lista para armazenar as características de todas as classes
all_features = []

# Loop através de cada classe
for c in classes:
    path = c + '/'
    for filename in os.listdir(path):
        if filename.endswith('.wav') or filename.endswith('.ogg'):
            file_path = os.path.join(path, filename)
            x, sr = librosa.load(file_path)
            mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=numero_de_mfccs)

            # Média das características MFCCs para a amostra
            mfccs_mean = np.mean(mfccs, axis=1)

            all_features.append(mfccs_mean)

# Converter a lista de características em um array numpy
all_features = np.array(all_features)

# Criar um único scaler para todas as amostras e normalizar
scaler = StandardScaler()
all_features_norm = scaler.fit_transform(all_features)

# Salvar a normalização dos 90 MFCCs em um arquivo
with open('scalers_Novo_Modelo_MLP_NormalizadoV3.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Definir rótulos
all_labels = []
for c in classes:
    all_labels.extend([c] * len(os.listdir(c)))

# Dividir o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(all_features_norm, all_labels, test_size=0.3, random_state=42)

# Criar um classificador MLP
clf = MLPClassifier(hidden_layer_sizes=(40,), max_iter=200, early_stopping=True, validation_fraction=0.3, verbose=True)

# Treinar o classificador
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar a precisão do classificador
accuracy = accuracy_score(y_test, y_pred)
print('Precisão do classificador: {:.2f}%'.format(accuracy * 100))

Iteration 1, loss = 1.43446653
Validation score: 0.282051
Iteration 2, loss = 1.32609776
Validation score: 0.333333
Iteration 3, loss = 1.22609182
Validation score: 0.384615
Iteration 4, loss = 1.13455828
Validation score: 0.410256
Iteration 5, loss = 1.05077504
Validation score: 0.487179
Iteration 6, loss = 0.97412864
Validation score: 0.564103
Iteration 7, loss = 0.90371848
Validation score: 0.666667
Iteration 8, loss = 0.83863691
Validation score: 0.692308
Iteration 9, loss = 0.77856148
Validation score: 0.717949
Iteration 10, loss = 0.72300693
Validation score: 0.717949
Iteration 11, loss = 0.67174396
Validation score: 0.743590
Iteration 12, loss = 0.62458263
Validation score: 0.769231
Iteration 13, loss = 0.58079878
Validation score: 0.769231
Iteration 14, loss = 0.54029607
Validation score: 0.794872
Iteration 15, loss = 0.50283255
Validation score: 0.820513
Iteration 16, loss = 0.46828271
Validation score: 0.820513
Iteration 17, loss = 0.43612799
Validation score: 0.846154
Iterat

In [8]:
#with open('scalers_Novo_Modelo_MLP_Normalizado.pkl', 'wb') as scalers_file:
#    pickle.dump(scaler, scalers_file)

modelo_filename = 'Novo_Modelo_MLP_NormalizadoV3.pkl'
with open(modelo_filename, 'wb') as model_file:
    pickle.dump(clf, model_file)

In [9]:
import pickle

# Carregar os Scalers
with open('scalers_Novo_Modelo_MLP_NormalizadoV3.pkl', 'rb') as scalers_file:
    scaler = pickle.load(scalers_file)

# Carregar o modelo treinado
modelo_filename = 'Novo_Modelo_MLP_NormalizadoV3.pkl'
with open(modelo_filename, 'rb') as model_file:
    clf = pickle.load(model_file)


In [13]:
# Carregar e pré-processar o novo áudio
new_audio_path = 'Laugh_LimpoV2/laugh_1.m4a_1_resampled.wav'  # Substitua pelo caminho do novo áudio
x, sr = librosa.load(new_audio_path)
mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=numero_de_mfccs)  # Use o mesmo número de MFCCs do treinamento
mfccs_mean = np.mean(mfccs, axis=1)
mfccs_norm = scaler.transform(mfccs_mean.reshape(1, -1))  # Aplicar normalização usando o scaler

# Fazer previsão com o modelo
predicted_class = clf.predict(mfccs_norm)

# O resultado 'predicted_class' contém o rótulo previsto para o novo áudio
print('Classe prevista:', predicted_class)

Classe prevista: ['Laugh_LimpoV3']
